In [22]:
from cw.io import read_cw_data, read_efas_data
from cw.cfg import DATA_DIR
import matplotlib.pylab as plt
import numpy as np
from haversine import haversine, Unit

# Clean the CrowdWater data

In [2]:
cw_data = read_cw_data()
print(f"Total number of readings are : {len(cw_data)}")
print(f"Total number of stations are : {cw_data.ROOT_ID.nunique()}")

Total number of readings are : 34415
Total number of stations are : 5999


/home/esowc31/CW4Floods/cw/io.py:45: DtypeWarning: Columns (2,3,7,11,16,28,36,38,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_name)


### Remove unneccessary column

In [3]:
keep = ['ROOT_ID', 'LATITUDE', 'LONGITUDE', 'WATER_LEVEL', 'SPOTTED_AT']
cw_data = cw_data.filter(keep)
cw_data

,ROOT_ID,LATITUDE,LONGITUDE,WATER_LEVEL,SPOTTED_AT
0,16726,47.029225,9.433322,NaN,2017-02-05 14:49
1,16726,47.029225,9.433322,NaN,2017-02-05 14:50
2,16736,47.398311,8.543917,NaN,2017-02-10 08:36
3,16737,47.560411,7.589407,NaN,2017-02-11 12:36
4,16740,47.12788,8.744036,NaN,2017-02-12 10:39
...,...,...,...,...,...
34410,233176,47.985418,7.959271,minus 3,2022-06-03 18:04
34411,628930,46.30159,7.870097,NaN,2022-06-04 13:07
34412,474150,47.201162,9.311755,NaN,2022-06-04 13:46
34413,402713,47.200459,9.311807,NaN,2022-06-04 13:47


In [4]:
cw_data = cw_data[cw_data.WATER_LEVEL.notna()]
cw_data = cw_data[cw_data.WATER_LEVEL != "false" ]


print(f"Total number of readings are : {cw_data.shape[0]}")
print(f"Total number of stations are : {cw_data.ROOT_ID.nunique()}")

Total number of readings are : 7320
Total number of stations are : 579


In [30]:
reading_frequency = np.unique(cw_data.ROOT_ID, return_counts = True)
sorted_freq = sorted(zip(reading_frequency[1], reading_frequency[0]), reverse=True)
tuples = zip(*sorted_freq)
freq, station_id = [ list(tuple) for tuple in tuples]

[923, 561, 451, 387, 292, 216, 182, 179, 162, 139, 118, 110, 102, 91, 89, 87, 79, 78, 76, 66, 65, 63, 60, 60, 55, 50, 46, 44, 40, 37, 36, 36, 34, 34, 33, 31, 30, 30, 29, 27, 27, 26, 25, 25, 25, 24, 24, 23, 23, 23, 20, 19, 18, 18, 18, 17, 16, 16, 16, 16, 16, 15, 15, 14, 14, 14, 14, 14, 13, 13, 12, 12, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 10, 10, 10, 10, 10, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [6]:
index = [i for i, val in enumerate(freq) if val>1]
station_id = [station_id[i] for i in index]

In [7]:
cw_data = cw_data[cw_data.ROOT_ID.isin(station_id)]
print(f"Total number of readings are : {cw_data.shape[0]}")
print(f"Total number of stations are : {cw_data.ROOT_ID.nunique()}")

Total number of readings are : 7125
Total number of stations are : 384


In [8]:
efas = read_efas_data(2020)
efas

<xarray.Dataset>
Dimensions:     (time: 366, y: 950, x: 1000)
Coordinates:
  * time        (time) datetime64[ns] 2020-01-01T06:00:00 ... 2020-12-31T06:0...
    step        timedelta64[ns] ...
    surface     float64 ...
    latitude    (y, x) float64 ...
    longitude   (y, x) float64 ...
    valid_time  (time) datetime64[ns] ...
Dimensions without coordinates: y, x
Data variables:
    dis24       (time, y, x) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2022-06-05T18:36 GRIB to CDM+CF via cfgrib-0.9.1...

# Distane Calculator 

In [28]:
distance_dict = dict()
mask = efas.dis24.mean(axis=0).values
n_station = 0

with open(DATA_DIR + 'station_ind.tsv', "w") as f:
    f.write(f"ROOT_ID\tMinimum Distance (KM)\tLat Index\tLon Index\n")

for index, row in cw_data.iloc[:2, :].iterrows():
    if row.ROOT_ID not in distance_dict:
        n_station+=1
        min_d = 2   
        for lat_index in range(900):
            for lon_index in range(1000):
                if np.isnan(mask[lat_index, lon_index]) == False :
                    with open(DATA_DIR + 'station_ind.tsv', "a") as f:

                        lat = efas.latitude.values[lat_index][lon_index]
                        lon = efas.longitude.values[lat_index][lon_index]
                        dist = haversine((float(row.LATITUDE), float(row.LONGITUDE)), (lat, lon), unit = Unit.KILOMETERS)
                                            
                        if dist < min_d:
                            min_d = dist
                            f.write(f"{row.ROOT_ID}\t{min_d:.3f}\t{lat_index}\t{lon_index}\n")
                            distance_dict[row.ROOT_ID] = (min_d, lat_index, lon_index)
                else:
                    pass
        
        print(f"{n_station} station discovered")
    else:
        pass

1 station discovered
2 station discovered


In [29]:
distance_dict

{16845: (1.7766167137011488, 557, 345), 16751: (1.3793539924282292, 558, 344)}

{1: (2, 3, 4)}

False

InvalidIndexError: (slice(None, 2, None), slice(None, None, None))